In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.stats import linregress

In [2]:
src = "../data"
dst = "../data"

# Process timelines

## Transform tweet table into URL table

In [21]:
fname = "combined_midterm_candidate_timelines_2022-01-01_to_2023-05-01_clean.csv.gzip"
cols = ["id", "author_id", "created_at", "expanded_urls", "retweeted", "quoted",
        "reply", "text", "retweet_count", "reply_count", "like_count",
        "quote_count"]
tweets = pd.read_csv(
    Path(src, fname),
    dtype={"id":str, "author_id":str},
    parse_dates=["created_at"],
    compression="gzip",
    usecols=cols
)

In [4]:
# parse the URL lists
tweets["expanded_urls"] = tweets["expanded_urls"].fillna("[]")
tweets["expanded_urls"] = tweets["expanded_urls"].apply(lambda x: eval(x))
tweets["has_url"] = tweets["expanded_urls"].apply(lambda x: len(x) > 0)

In [5]:
tweets["N_urls"] = tweets["expanded_urls"].apply(lambda x: len(x))

In [6]:
# expand only entries with multiple URLs
multiple_urls = tweets[tweets["N_urls"] > 1]
expanded_urls = pd.DataFrame()
for idx, entry in multiple_urls.iterrows():
    row = {key:val for key, val in entry.items()}
    expanded_urls = pd.concat([expanded_urls, pd.DataFrame(row)])
    
expanded_urls = expanded_urls.set_index("id")
urls = tweets.copy()
urls = urls.set_index("id")
# drop entries with mutiple URLs
urls = urls.drop(multiple_urls["id"].values)
# add expanded entries with one line for each URL
urls = pd.concat([urls, expanded_urls])
urls = urls.reset_index()

In [7]:
len(tweets)

1084776

In [8]:
len(urls)

1212940

In [9]:
# now, some URLs are stored as singular entries of a list, and some as string.
# empty entries are stored as empty list. Below we streamline URL entries such
# that every entry is a single string
def extract_URL_from_list(entry):
    if len(entry) == 0:
        return np.nan
    elif len(entry) == 1:
        return entry[0]
    else:
        return entry
    
urls["expanded_urls"] = urls["expanded_urls"].apply(extract_URL_from_list)
urls = urls.rename(columns={"expanded_urls":"url"})

In [10]:
# some tweets contain the same URL twice. We drop these
N = len(urls)
urls = urls.drop_duplicates(subset=["id", "url"])
print(f"dropped {N - len(urls)} duplicate URL entries")

dropped 85207 duplicate URL entries


In [22]:
del tweets

In [12]:
# save the outcome
fname = "combined_midterm_candidate_timelines_2022-01-01_to_2023-05-01_clean_urls.csv.gzip"
urls.to_csv(Path(dst, fname), compression="gzip", index=False)

In [15]:
# load the data frame with the expanded URLs
fname = "combined_midterm_candidate_timelines_2022-01-01_to_2023-05-01_clean_urls.csv.gzip"
cols = ["id", "author_id", "created_at", "url", "retweeted",
        "quoted", "reply", "has_url"]
urls = pd.read_csv(
    Path(src, fname),
    compression="gzip",
    usecols=cols,
    parse_dates=["created_at"],
    dtype={"author_id":str, "id":str}
)

In [23]:
# add handle back to data frame
fname = "candidate_twitter_profiles.csv"
cols = ["author_id", "handle"]
users = pd.read_csv(
    Path(src, fname),
    dtype={"author_id":str},
    usecols=cols
)

In [26]:
urls = pd.merge(
    urls,
    users,
    how="left",
    left_on="author_id",
    right_on="author_id"
)

In [29]:
del users

## Add engagement metrics

In [30]:
# load the public metrics information for the collected tweets
fname = "combined_midterm_candidate_timelines_2022-01-01_to_2023-05-01_clean.csv.gzip"
tweet_metrics = pd.read_csv(Path(src, fname),
                 compression="gzip",
                 usecols=["id", "retweet_count",
                          "reply_count", "like_count", "quote_count"],
                dtype={"id":str})
tweet_metrics = tweet_metrics.drop_duplicates(subset="id")
# merge the tweet metrics with the tweet data frame
urls = pd.merge(urls, tweet_metrics, how="left", left_on="id", right_on="id")
del tweet_metrics

## Add unraveled URLs

In [31]:
# load the list of originally shortened URLs with their expansions to their true
# destination
fname = "midterm_candidates_unraveled_urls.csv.xz"
unraveled_urls = pd.read_csv(
    Path(src, fname), 
    compression="xz",
    usecols=["url", "unraveled_url"]
)

In [32]:
# add URL information
urls = pd.merge(
    urls,
    unraveled_urls[["url", "unraveled_url"]],
    how="left",
    left_on="url",
    right_on="url"
)

# add indicator of whether the URL was originally shortened
urls["shortened_url"] = False
urls.loc[urls["unraveled_url"].dropna().index, "shortened_url"] = True

# replace the shortened URL with the unraveled URL
urls.loc[urls["unraveled_url"].dropna().index, "url"] = \
    urls.loc[urls["unraveled_url"].dropna().index, "unraveled_url"]
urls = urls.drop(columns=["unraveled_url"])

In [33]:
# extract the domain from the URL. Note: a few "found malformed URL" warnings
# are acceptable
def extract_domain(url):
    '''Given an ULR, extracts the domain name in the form XXXXX.YY'''
    if url != url:
        return np.nan
    # reformat entries that have the domain after a general name in parantheses
    if url.find('(') > 0:
        url = url.split('(')[-1]
        url = url.strip(')')
    # trailing "/" and spaces
    url = url.strip('/').strip()
    # transform all domains to lowercase
    url = url.lower()
    # remove any white spaces
    url = url.replace(' ', '')
    # if present: remove the protocol
    if url.startswith(("http", "https")):
        try:
            url = url.split('//')[1]
        except IndexError:
            print(f"found malformed URL {url}")
            return np.nan
    # remove "www." 
    url = url.replace('www.', '')
    url = url.split("/")[0]
    return url

urls["domain"] = urls["url"].apply(extract_domain)

## Add NewsGuard nutrition scores

Newsguard rating threshold to label a domain as "untrustworthy": 60 (see [description](https://www.newsguardtech.com/ratings/rating-process-criteria/)).

In [34]:
threshold = 60

In [35]:
# load the nutrition labels from the day of the midterm election: 2022-11-08
fname = "metadata-2022110801.csv"
cols = ["Domain", "Score", "Last Updated"]
NG_scores = pd.read_csv(Path(src, fname), usecols=cols)
# if more than one score exists for the same domain, keep the most recent one
NG_scores = NG_scores.sort_values(by=["Domain","Last Updated"], ascending=False)
NG_scores = NG_scores.drop_duplicates(subset=["Domain"])
NG_scores = NG_scores.rename(columns={"Domain":"domain", "Score":"NG_score"})
NG_scores = NG_scores.drop(columns=["Last Updated"])

# threshold scores to define untrustworthy domains
NG_scores["NG_untrustworthy"] = 0
NG_scores.loc[NG_scores[NG_scores["NG_score"] < threshold].index, "NG_untrustworthy"] = 1

# add the nutrition information to the tweet data table
urls = pd.merge(urls, NG_scores,
         left_on="domain", right_on="domain", how="left")
del NG_scores

## Tweet length

In [36]:
# load the cleaned timeline-data
fname = "combined_midterm_candidate_timelines_2022-01-01_to_2023-05-01_clean.csv.gzip"
cols = ["id", "text"]
texts = pd.read_csv(
    Path(src, fname),
    compression="gzip",
    usecols=cols,
    dtype={"id":str, "text":str}
)

texts["tweet_length"] = texts["text"].apply(lambda x: len(x))

urls = pd.merge(
    urls,
    texts[["id", "tweet_length"]],
    how="left",
    left_on="id",
    right_on="id"
)

## Add belief-speaking and fact-speaking scores

In [37]:
# load the embedding scores for belief-speaking and truth-seeking
fname = "combined_midterm_candidate_timelines_2022-01-01_to_2023-05-01_honesty_component_scores.csv.gzip"
honesty_scores = pd.read_csv(
    Path(src, fname),
    dtype={"id":str}, 
    compression="gzip"
).rename(columns={"avg_fact_score":"avg_fact_score_raw", "avg_belief_score":"avg_belief_score_raw"})

In [38]:
honesty_scores = pd.merge(
    honesty_scores,
    urls[["id", "tweet_length"]],
    how="left",
    left_on="id",
    right_on="id"
)
honesty_scores = honesty_scores.dropna()

In [39]:
# correct the similarity scores for tweet-length effects
slope_belief, intercept_belief, rval_belief, pval_belief, stderr_belief = \
    linregress(honesty_scores["tweet_length"], honesty_scores["avg_belief_score_raw"])
print(f"belief-speaking slope: {slope_belief}, intercept: {intercept_belief}")

def predict_belief_similarity(tweet_length):
    return intercept_belief + slope_belief * tweet_length

slope_fact, intercept_fact, rval_fact, pval_fact, stderr_fact = \
    linregress(honesty_scores["tweet_length"], honesty_scores["avg_fact_score_raw"])
print(f"fact-seeking slope: {slope_fact}, intercept: {intercept_fact}")

def predict_fact_similarity(tweet_length):
    return intercept_fact + slope_fact * tweet_length

honesty_scores["avg_belief_score"] = honesty_scores\
    .apply(lambda x: x["avg_belief_score_raw"] - predict_belief_similarity(x["tweet_length"]), axis=1)

honesty_scores["avg_fact_score"] = honesty_scores\
    .apply(lambda x: x["avg_fact_score_raw"] - predict_fact_similarity(x["tweet_length"]), axis=1)

belief-speaking slope: 0.00021486802643395315, intercept: 0.6849102858687011
fact-seeking slope: 0.00032929502778065143, intercept: 0.5772773113543177


In [40]:
# merge the raw and corrected scores with the url data frame
cols = ["id", "avg_fact_score_raw", "avg_fact_score", "avg_belief_score_raw", "avg_belief_score"]
urls = pd.merge(
    honesty_scores[cols], 
    urls, 
    how="right", 
    left_on="id", 
    right_on="id"
)

## Create a tweet data frame

In [41]:
tweet_cols = ["id", "author_id", "handle", "created_at", "retweeted", "quoted", "reply",
              "has_url", "tweet_length", "avg_belief_score", "avg_fact_score"]
tweets = urls[tweet_cols].drop_duplicates(subset=["id"]).copy()

In [42]:
# Calculate average NewsGuard score
average_scores = urls[["id", "NG_score"]]\
    .groupby("id")\
    .agg("mean")

average_scores["NG_unreliable"] = np.nan
average_scores.loc[average_scores[\
            average_scores["NG_score"] < 60].index, "NG_unreliable"] = 1
average_scores.loc[average_scores[\
            average_scores["NG_score"] >= 60].index, "NG_unreliable"] = 0

tweets = pd.merge(tweets, average_scores, how="left", left_on="id", right_on="id")
del average_scores

## Create a user data frame

In [43]:
users = tweets[["author_id", "handle", "id"]]\
    .groupby(["author_id", "handle"])\
    .agg("count")\
    .reset_index()\
    .rename(columns={"id":"N_tweets"})

### Add account stats

In [44]:
fname = "candidate_twitter_profiles.csv"
cols = ["followers_count", "following_count", "tweet_count", "created_at", 
        "author_id"]
account_stats = pd.read_csv(
    Path(src, fname),
    parse_dates=["created_at"],
    usecols=cols,
    dtype={"author_id":str}
)

users = pd.merge(users, account_stats, how="left", left_on="author_id", right_on="author_id")
del account_stats

### Add share of untrustworthy domains (NewsGuard)

In [45]:
cols = ["author_id", "NG_unreliable"]
unreliable_user_count = tweets[tweets["retweeted"] == False][cols]\
    .groupby("author_id")\
    .agg(["sum", "count"])

unreliable_user_count["NG_unreliable_share"] = \
    unreliable_user_count["NG_unreliable"]["sum"] / \
    unreliable_user_count["NG_unreliable"]["count"]
    
# flatten the hierarchical indices
unreliable_user_count = unreliable_user_count.reset_index()
unreliable_user_count.columns = ['_'.join(col).strip("_") \
                            for col in unreliable_user_count.columns.values]

unreliable_user_count.head(2)

,author_id,NG_unreliable_sum,NG_unreliable_count,NG_unreliable_share
0,100049648,1.0,4,0.25
1,1002275817227980800,0.0,13,0.00


In [46]:
cols = ["NG_unreliable_share", "author_id"]
users = pd.merge(
    users, 
    unreliable_user_count[cols],
    how="left",
    left_on="author_id",
    right_on="author_id"
)

### Add average NewsGuard score

In [47]:
average_NG_scores = tweets[tweets["retweeted"] == False][["author_id", "NG_score"]]\
    .groupby("author_id")\
    .mean()\
    .reset_index()\
    .rename(columns={"NG_score":"NG_score_mean"})
users = pd.merge(
    users, 
    average_NG_scores, 
    how="left", 
    left_on="author_id", 
    right_on="author_id"
)

## Data exports

In [48]:
# URL data frame
fname = "midterm_URLs_2022-01-01_to_2023-05-01.csv.gzip"
urls = urls[urls["has_url"] == True]
urls = urls.drop(columns=["url", "has_url"])
urls.to_csv(Path(dst, fname), index=False, compression="gzip")

In [52]:
# user data frame
fname = "midterm_users_2022-01-01_to_2023-05-01.csv"
users.to_csv(Path(dst, fname), index=False)

In [51]:
# tweet data frame
fname = "midterm_tweets_2022-01-01_to_2023-05-01.csv.gzip"
tweets.to_csv(Path(dst, fname), index=False, compression="gzip")